# Sketchmap Demo

In [ ]:
import ipywidgets as widgets
from IPython.display import display
from subprocess_widget import SubprocessWidget
from bqplot import pyplot as plt
import numpy as np

In [ ]:
from subprocess_widget import SubprocessWidget

In [ ]:
w = SubprocessWidget(cmd=["./create_new_sketchmap.sh", "traj.lys.dipeptide.bare"], log_fn="sketchmap.log")
display(w)

In [ ]:
out_fn = "traj.lys.dipeptide.bare-n10-l8-c3.5-g0.5_fastavg_nnrm.OOS.colvar"
a = np.loadtxt(out_fn)

plt.figure()
plt.scatter(a[:,0], a[:,1], color=a[:,2])
plt.show()